In [1]:
import pandas as pd
data = pd.read_excel("https://raw.githubusercontent.com/estebanrucan/bases-de-datos/main/mcdonalds.xlsx").iloc[:, 2:]
data.head()

,categoria,tamano_porcion,calorias,calorias_de_grasa,grasa_total,grasa_total_pct_diario,grasa_saturada,grasa_saturada_pct_diario,grasas_trans,colesterol,...,carbohidratos,carbohidratos_pct_diario,fibra_dietetica,fibra_dietetica_pct_diario,azucares,proteina,vitamina_a_pct_diario,vitamina_c_pct_diario,calcio_pct_diario,hierro_pct_diario
0,Desayuno,136,300,120,13.0,20,5.0,25.0,0.0,260,...,31,10,4,17,3,17,10,0,25,15
1,Desayuno,135,250,70,8.0,12,3.0,15.0,0.0,25,...,30,10,4,17,3,18,6,0,25,8
2,Desayuno,111,370,200,23.0,35,8.0,42.0,0.0,45,...,29,10,4,17,2,14,8,0,25,10
3,Desayuno,161,450,250,28.0,43,10.0,52.0,0.0,285,...,30,10,4,17,2,21,15,0,30,15
4,Desayuno,161,400,210,23.0,35,8.0,42.0,0.0,50,...,30,10,4,17,2,21,6,0,25,10


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import statsmodels.api as sm

In [4]:
ms = sm.OLS.from_formula("calorias ~ 1", data).fit()
mb = sm.OLS.from_formula("calorias ~ tamano_porcion", data).fit()

In [6]:
from estyp.testing import nested_models_test

nested_models_test(ms, mb, data["calorias"])


    Nested models F-test
    --------------------
    F = 222.8743 | df: {'df_num': '1.00', 'df_den': '119.00'} | p-value = <0.0001
    alternative hypothesis: big model is true
    sample estimates:
      Difference in deviances between models: 4962297.843197
    

In [158]:
ms.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               calorias   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.364e+04
Date:                Wed, 26 Jul 2023   Prob (F-statistic):          1.78e-187
Time:                        13:50:36   Log-Likelihood:                -321.52
No. Observations:                 115   AIC:                             661.0
Df Residuals:                     106   BIC:                             685.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     0.4496      0.940      0.478      0.633      -1.414       2.314
grasa_total                   3.7845      1.149      3.292      0.001       1.506       6.063
carbohidratos                 5.1574      0.497     10.380      0.000       4.172       6.142
proteina                      3.9876      0.059     67.514      0.000       3.870       4.105
grasa_total_pct_diario        3.2836      0.747      4.397      0.000       1.803       4.764
calcio_pct_diario            -0.2442      0.060     -4.059      0.000      -0.363      -0.125
grasa_saturada_pct_diario     0.0864      0.036      2.413      0.018       0.015       0.157
carbohidratos_pct_diario     -3.3141      1.497     -2.213      0.029      -6.283      -0.345
grasas_trans                  2.0056      1.036      1.936      0.056      -0.048       4.059
==============================================================================
Omnibus:                        5.672   Durbin-Watson:                   1.807
Prob(Omnibus):                  0.059   Jarque-Bera (JB):                6.999
Skew:                          -0.241   Prob(JB):                       0.0302
Kurtosis:                       4.108   Cond. No.                         381.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [153]:
m_actual.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               calorias   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     575.4
Date:                Wed, 26 Jul 2023   Prob (F-statistic):           1.20e-59
Time:                        13:48:25   Log-Likelihood:                -659.83
No. Observations:                 115   AIC:                             1326.
Df Residuals:                     112   BIC:                             1334.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                   105.7719     13.320      7.941      0.000      79.379     132.164
grasa_saturada_pct_diario     1.1198      0.485      2.307      0.023       0.158       2.081
calorias_de_grasa             1.5610      0.088     17.795      0.000       1.387       1.735
==============================================================================
Omnibus:                        4.390   Durbin-Watson:                   0.745
Prob(Omnibus):                  0.111   Jarque-Bera (JB):                4.056
Skew:                           0.458   Prob(JB):                        0.132
Kurtosis:                       3.083   Cond. No.                         467.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

1892.3736101338984

In [64]:
isinstance(m, sm.OLS)

False

In [65]:
m

'calorias ~ grasa_total + carbohidratos + proteina + categoria + grasa_total_pct_diario + fibra_dietetica_pct_diario + carbohidratos_pct_diario'

In [31]:
modelo_actual

'calorias ~ 1 + grasa_total + carbohidratos + proteina + categoria + grasa_total_pct_diario + fibra_dietetica_pct_diario + carbohidratos_pct_diario'

In [20]:
at.loc[p][-1]

1.121518264624544e-16